In [2]:
import sys
import os

# Chemin absolu du répertoire racine du projet
project_root = '/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/'

# Ajouter le répertoire src au chemin de recherche des modules
src = os.path.join(project_root, '.')
sys.path.append(src)



In [3]:
import pandas as pd
import numpy as np
from src.data_concatenation.concatenate import build_macro_table
from src.data_preprocessing.preprocessing import preprocess_dataframe
from src.data_preprocessing.topificator import TopicExtractor
from src.database_management.database_initializer import DatabaseInitializer, wait_for_postgresql
from src.database_management.dataframe_to_transactionalDB import insert_data_from_dataframe
from src.database_management.migrate_to_decisionalDB import migration_to_decisionalDB
from src.database_management.db_models import Base
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from datetime import datetime, timedelta
import psycopg2, os

from tqdm import tqdm

CURRENT_DATE = datetime.now().strftime("%Y-%m-%d")
DATA_PATH = os.path.join(project_root, 'data')
CITIES_PATH = os.path.join(project_root, 'data', 'utils', 'countries_cities-full.json')
RAW_SAVE_PATH = os.path.join(project_root, 'data', 'temp')
PARCKET_PATH = os.path.join(project_root, 'data', 'parquet', CURRENT_DATE)

In [3]:

# --------------------------------------------------------
# 0.3 Preprocessing of concatenate parquets
# --------------------------------------------------------
# df = preprocess_dataframe(concatenate_parquets)

In [4]:
# all df
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# df


In [5]:
bank_reviews = pd.read_csv(os.path.join(project_root, 'data', 'processed', '2024-08-11', 'macro_llamma.csv'))

In [16]:
bank_reviews

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Reviewer_Name,Reviewer_Star,Reviewer_Text,Reviewer_Publish_Date,Reviewer_Like_Reaction,Reviewer_Profile_Link,Reviewer_Owner_Reply,Reviewer_Owner_Reply_Date,Topic,Sentiment,Sub_Topic
0,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Hamza Izourane,4,"L'emplacement est bien situé, il est facile de...",2018-08-13 00:58:13.786458,0,https://www.google.com/maps/contrib/1071099367...,Nothing,1970-01-01 00:00:00.000000,Localisation et accessibilité,Positive,"['emplacement bien situé', 'facile de trouver ..."
1,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Hamza Izourane,4,"L'emplacement est bien situé, il est facile de...",2018-08-13 00:58:13.786458,0,https://www.google.com/maps/contrib/1071099367...,Nothing,1970-01-01 00:00:00.000000,Expérience utilisateur,Positive,['employés traitent bien les clients']
2,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Hamza Izourane,4,"L'emplacement est bien situé, il est facile de...",2018-08-13 00:58:13.786458,0,https://www.google.com/maps/contrib/1071099367...,Nothing,1970-01-01 00:00:00.000000,Services additionnels,Positive,['guichets automatiques disponibles']
3,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Imjou Fad,5,"Très bon accueil, le vigile très serviable et ...",2023-08-12 00:58:13.786474,0,https://www.google.com/maps/contrib/1142323693...,Nothing,1970-01-01 00:00:00.000000,Service client,Positive,"['Accueil bienvenu', 'Vigile serviable et sour..."
4,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Mobi Sud,4,La Banque Au Cheval,2017-08-13 00:58:13.786484,1,https://www.google.com/maps/contrib/1013145203...,Nothing,1970-01-01 00:00:00.000000,Frais bancaires,Negative,['élevés']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26158,Senegal,Touba,Touba Transfert Multi Services,+221 77 633 23 33,"Touba Mousque, Tiouba Mosquee 22111, Sénégal",not available,Cheikh Ndiaye,5,Très bonne expérience,2020-08-12 00:58:13.851055,0,https://www.google.com/maps/contrib/1151922927...,Nothing,1970-01-01 00:00:00.000000,satisfaction client,Positive,['très satisfait']
26159,Senegal,Touba,Microcred,Not available,"R4R7+C83, N3, Touba, Sénégal",not available,Mamane Cisse,5,Merci,2017-08-13 00:58:13.851057,0,https://www.google.com/maps/contrib/1152507324...,Nothing,1970-01-01 00:00:00.000000,Service client,Positive,"['Rapide', 'Simple']"
26160,Senegal,Touba,Microcred,Not available,"R4R7+C83, N3, Touba, Sénégal",not available,Mamadou Cissé,5,Espèces,2018-08-13 00:58:13.851060,1,https://www.google.com/maps/contrib/1038651926...,Nothing,1970-01-01 00:00:00.000000,services de dépôt,Negative,"['dificiles', 'complexes']"
26161,Senegal,Touba,Smk,Not available,"V439+Cm5, Touba, Sénégal",not available,Fadla Services,4,Bonne expérience,2021-08-12 00:58:13.851069,0,https://www.google.com/maps/contrib/1171719169...,Nothing,1970-01-01 00:00:00.000000,Service client,Positive,"['réactif', 'rapidité']"


In [6]:
# number of duppliacted rows
bank_reviews.duplicated().sum()
# drop duplicates
bank_reviews.drop_duplicates(inplace=True)

In [7]:
# find row where Reviewer_Name	Reviewer_Star	Reviewer_Text	Reviewer_Publish_Date	is same
bank_reviews.duplicated(subset=['Reviewer_Name', 'Reviewer_Star', 'Reviewer_Text', 'Reviewer_Publish_Date']).sum()

8686

In [8]:
# drop duplicates
bank_reviews.drop_duplicates(subset=['Reviewer_Name', 'Reviewer_Star', 'Reviewer_Text', 'Reviewer_Publish_Date'], inplace=True)

In [9]:
bank_reviews

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Reviewer_Name,Reviewer_Star,Reviewer_Text,Reviewer_Publish_Date,Reviewer_Like_Reaction,Reviewer_Profile_Link,Reviewer_Owner_Reply,Reviewer_Owner_Reply_Date,Topic,Sentiment,Sub_Topic
0,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Hamza Izourane,4,"L'emplacement est bien situé, il est facile de...",2018-08-13 00:58:13.786458,0,https://www.google.com/maps/contrib/1071099367...,Nothing,1970-01-01 00:00:00.000000,Localisation et accessibilité,Positive,"['emplacement bien situé', 'facile de trouver ..."
3,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Imjou Fad,5,"Très bon accueil, le vigile très serviable et ...",2023-08-12 00:58:13.786474,0,https://www.google.com/maps/contrib/1142323693...,Nothing,1970-01-01 00:00:00.000000,Service client,Positive,"['Accueil bienvenu', 'Vigile serviable et sour..."
4,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Mobi Sud,4,La Banque Au Cheval,2017-08-13 00:58:13.786484,1,https://www.google.com/maps/contrib/1013145203...,Nothing,1970-01-01 00:00:00.000000,Frais bancaires,Negative,['élevés']
6,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Patxi,4,Il y a peu de guichets automatiques dans la ré...,2021-08-12 00:58:13.786488,1,https://www.google.com/maps/contrib/1006364945...,Nothing,1970-01-01 00:00:00.000000,guichets automatiques,Positive,"['fonctionnels', 'abondants']"
8,Morocco,Taroudant,Banque Populaire,+212 5288-52588,"F4Fm+H48, N10, Taroudant",http://www.gbp.ma/,Abdel Amah,5,Merci beaucoup pour votre excellent service.,2019-08-13 00:58:13.786492,1,https://www.google.com/maps/contrib/1124624498...,Nothing,1970-01-01 00:00:00.000000,Satisfaction client,Positive,['excellent service']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26156,Senegal,Touba,Touba Transfert Multi Services,+221 77 633 23 33,"Touba Mousque, Tiouba Mosquee 22111, Sénégal",not available,Bernard Diouf,1,Très mauvaise expérience,2021-08-12 00:58:13.851052,0,https://www.google.com/maps/contrib/1104427616...,Nothing,1970-01-01 00:00:00.000000,Expérience utilisateur,Negative,['Mauvaise expérience']
26157,Senegal,Touba,Touba Transfert Multi Services,+221 77 633 23 33,"Touba Mousque, Tiouba Mosquee 22111, Sénégal",not available,Cheikh Ndiaye,5,Très bonne expérience,2020-08-12 00:58:13.851055,0,https://www.google.com/maps/contrib/1151922927...,Nothing,1970-01-01 00:00:00.000000,service clientèle,Positive,"['réactif', 'rapide']"
26159,Senegal,Touba,Microcred,Not available,"R4R7+C83, N3, Touba, Sénégal",not available,Mamane Cisse,5,Merci,2017-08-13 00:58:13.851057,0,https://www.google.com/maps/contrib/1152507324...,Nothing,1970-01-01 00:00:00.000000,Service client,Positive,"['Rapide', 'Simple']"
26160,Senegal,Touba,Microcred,Not available,"R4R7+C83, N3, Touba, Sénégal",not available,Mamadou Cissé,5,Espèces,2018-08-13 00:58:13.851060,1,https://www.google.com/maps/contrib/1038651926...,Nothing,1970-01-01 00:00:00.000000,services de dépôt,Negative,"['dificiles', 'complexes']"


In [ ]:

# --------------------------------------------------------
# 0.5 Initialization of database
# --------------------------------------------------------
# Utilisation de la DatabaseInitializer
ADMIN_USER = 'postgres'
ADMIN_PASSWORD = 'postgres'
HOST = 'localhost'
PORT = '5432'
DB_USER = 'conite'
DB_PASSWORD = 'conite_password'
DB_NAME = 'bank_reviews'

# --------------------------------------------------------------------------
# Database Initialization
# --------------------------------------------------------------------------
db_initializer = DatabaseInitializer(ADMIN_USER, ADMIN_PASSWORD, HOST, PORT)

db_initializer.create_database_and_user(DB_USER, DB_PASSWORD, DB_NAME)

# ------------------------------------------------------------------------------
# Configuration de la base de donnees
# ------------------------------------------------------------------------------
DB_URI = f'postgresql://{DB_USER}:{DB_PASSWORD}@{HOST}:5432/{DB_NAME}'

# ------------------------------------------------------------------------------
# Initialize database and create tables
# ------------------------------------------------------------------------------

engine = create_engine(DB_URI)
Session = sessionmaker(bind=engine)
session = Session()
wait_for_postgresql(host=HOST, port=PORT, user=DB_USER, password=DB_PASSWORD, dbname=DB_NAME)
Base.metadata.create_all(engine)

conn_params = {
    'dbname': DB_NAME,
    'user': DB_USER,
    'password': DB_PASSWORD,
    'host': HOST,
    'port': PORT
}
with psycopg2.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        """
        cur.execute(query)
        existing_tables = cur.fetchall()
        existing_tables = [table[0] for table in existing_tables]
        print("\033[1;92mTABLES CREATED SUCCESSFULLY.\033[0m \033[1;94m" + str(existing_tables) + "\033[0m")

# --------------------------------------------------------
# 0.6 Insertion of data in database
# --------------------------------------------------------
insert_data_from_dataframe(bank_reviews, session)
migration_to_decisionalDB(DB_NAME, DB_PASSWORD, session)
# session.commit()

In [7]:
!python main.py

User conite already exists.
Database bank_reviews already exists.
Granted all privileges on database bank_reviews to user conite.
Granted all privileges on schema public to user conite.
User conite already exists.
Database decisional_db already exists.
Granted all privileges on database decisional_db to user conite.
Granted all privileges on schema public to user conite.
-------------------PostgreSQL is ready -------------------
TABLES CREATED SUCCESSFULLY. ['topics', 'countries', 'towns', 'sub_topics', 'banks', 'reviewers', 'reviews', 'sentiments']
DATA INSERTED SUCCESSFULLY.
/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/src/database_management/migrate_to_decisionalDB.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countries = pd.read_sql('SELECT * FROM countries', connection)
Traceback (most 

In [28]:
# !pip install python-dotenv

In [6]:
# !pip install sqlalchemy pandas
!pip install xmanager

  Using cached xmanager-0.5.0-py3-none-any.whl.metadata (12 kB)
  Using cached SQLAlchemy-1.2.19-cp311-cp311-linux_x86_64.whl
Using cached xmanager-0.5.0-py3-none-any.whl (210 kB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.32
    Uninstalling SQLAlchemy-2.0.32:
      Successfully uninstalled SQLAlchemy-2.0.32
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.0 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.2.19 which is incompatible.
langchain-community 0.0.9 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.2.19 which is incompatible.
